In [112]:
import re
import pandas as pd




In [113]:
# Load the CSV file
file_path = 'filename.csv'  # Thay đường dẫn với đường dẫn file CSV của bạn
df_input = pd.read_csv(file_path)



In [114]:

# Function to extract phone details
def extract_phone_details(item):
    # Loại bỏ bất kỳ phần nào nằm trong dấu ngoặc đơn
    item = re.sub(r"\(.*?\)", "", item).strip()

    # Extract core phone name (ignoring storage, condition, and connection types like 4G, Wifi, etc.)
    name = re.search(r"^(.*?)(\d+GB|\d+TB|\s\d+G|\s\d+GB|cũ|mới|chính hãng|\d{2}%)", item, re.IGNORECASE)
    name = name.group(1).strip() if name else item

    condition = re.search(r"(Cũ.*|Mới.*)", item, re.IGNORECASE)
    if condition:
        condition = condition.group(0).strip()
    else:
        # If no "Cũ" or "Mới", extract everything after "-"
        condition = re.search(r"-\s*(.*)", item)
        condition = condition.group(1).strip() if condition else None


    return name, condition




In [116]:
# Loop through each item in the 'Tên sản phẩm' column and extract details
# Create empty lists to store separated data
phone_names = []
conditions = []
for item in df_input['Tên sản phẩm']:
    name, condition = extract_phone_details(item)
    phone_names.append(name)
    conditions.append(condition)



df_output = pd.DataFrame({
    'brand':df_input['Thương hiệu'],
    'phone_name': phone_names,
    'condition1': conditions,
    'condition5':df_input['Tình trạng SP'],
    'condition3':df_input['Tình trạng sản phẩm'],
    'condition4':df_input['Tình trạng máy'],
    'oldprice':df_input['Giá bán cũ'],
    'newprice':df_input['Giá mới'],
    'ram':df_input['RAM'],
    'rom':df_input['Bộ nhớ trong'] 
})
def merge_conditions(row):
    conditions = [str(row[col]) for col in ['condition5', 'condition3', 'condition4'] if pd.notna(row[col])]
    return ', '.join(conditions) if conditions else None

# Áp dụng hàm để tạo cột condition mới
df_output['condition2'] = df_output.apply(merge_conditions, axis=1)

# Xóa các cột cũ sau khi hợp nhất
df_output.drop(['condition5', 'condition3', 'condition4'], axis=1, inplace=True)

In [117]:


# Save the modified dataframe back to CSV
output_file_path = 'newfilename.csv'  # Đặt tên file mà bạn muốn lưu
df_output.to_csv(output_file_path, index=False,encoding='utf-8')
print(f'Updated file saved at: {output_file_path}')

Updated file saved at: clickclean.csv
